# needed imports
from transformers import AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import torch, langchain, faiss, chainlit, pymupdf, docutils
#loading model
    from transformers import AutoModelForCausalLM
    model_name = "gpt2"  # This is the "path" or identifier
    model = AutoModelForCausalLM.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained('gpt2')
device = torch.device('cpu')
model.to(device)
# load encyclopedia
doc = pymupdf.open("/storage/emulated/0/Download/The-Gale-Encyclopedia_medicaldiagnosis.pdf")
text = doc.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512)
chunks = text_splitter.split_text(text)
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)
index = faiss.IndexFlatL2(embedding_size)
index.add(embeddings)
#prompt engineering
prompt_template = "Q: {question}\nA: {answer}"
from langchain.chains import RetrievalQAChain
retrieval_qa = RetrievalQAChain(
    llm=model,
    vectorstore=index,
    prompt_template=prompt_template
)
from chainlit import run
run(retrieval_qa)